IMPORT LIBRARIES

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
from sklearn.preprocessing import StandardScaler

np.random.seed(42)

# Create folder for raw files
os.makedirs("raw_data", exist_ok=True)


CREATE RAW DATA (3 CAMPUSES WITH INTENTIONAL ISSUES)

In [25]:
def create_students(campus_id, campus_name):
    df = pd.DataFrame({
        "Student_ID": [f"{campus_id}{i:03d}" for i in range(1,51)],
        "Full_Name": np.random.choice(["alice  ", "BOB@", "charles#", "Diane "], 50),
        "Gender": np.random.choice(["Male","female",None], 50),
        "DOB": np.random.choice(["2000/01/01","01-02-2001","2003-05-12",None], 50),
        "Program": np.random.choice(["ict ","Engineering"," BUSINESS"], 50),
        "Level": np.random.choice(["Level7","Level 6","level 7"], 50),
        "Intake_Year": np.random.choice([2020,2021,2022], 50),
        "Phone": np.random.choice(["0781234567","07A345678",None], 50)
    })
    
    df["Campus_ID"] = campus_id
    df["Campus_Name"] = campus_name
    df["Source_Campus_File"] = f"{campus_name.lower()}_students.csv"
    df["Upload_Date"] = datetime.today()
    
    return df

huye_students = create_students("HU","Huye")
kigali_students = create_students("KI","Kigali")
musanze_students = create_students("MU","Musanze")

students_all = pd.concat([huye_students,kigali_students,musanze_students])

students_all.to_csv("raw_data/students_raw.csv", index=False)


CREATE COURSES (WITH INCONSISTENCIES)

In [26]:
courses_all = pd.DataFrame({
    "Course_Code": ["CS101","CS-101","CS102 ","cs103"],
    "Course_Title": ["Python","Python","Databases","AI"],
    "Credits": [3,3,4,-2]  # negative credit issue
})

courses_all.to_csv("raw_data/courses_raw.csv", index=False)


CREATE RESULTS (WITH OUTLIERS & MISSING)

In [27]:
results_all = pd.DataFrame({
    "Student_ID": np.random.choice(students_all["Student_ID"], 200),
    "Course_Code": np.random.choice(courses_all["Course_Code"], 200),
    "Assessment_Type": np.random.choice(["Exam","Test","Assignment"], 200),
    "Mark": np.random.choice([45,60,75,110,-5,None], 200),
    "Assessment_Date": np.random.choice(["2023-01-10","10/02/2023","2023/03/15"], 200),
    "Academic_Year": "2023/2024",
    "Semester": np.random.choice(["Semester 1","Semester1"], 200),
    "Attendance_Rate": np.random.choice([40,60,80,105,None], 200)
})

results_all.to_csv("raw_data/results_raw.csv", index=False)


LOAD RAW DATA

In [48]:
students = pd.read_csv("raw_data/students_raw.csv")
courses = pd.read_csv("raw_data/courses_raw.csv")
results = pd.read_csv("raw_data/results_raw.csv")

print(students.shape, courses.shape, results.shape)
students.head()

(150, 12) (4, 3) (200, 8)


,Student_ID,Full_Name,Gender,DOB,Program,Level,Intake_Year,Phone,Campus_ID,Campus_Name,Source_Campus_File,Upload_Date
0,HU001,charles#,NaN,NaN,ict,Level 6,2020,0781234567,HU,Huye,huye_students.csv,2026-02-12 09:44:47.451886
1,HU002,Diane,female,2000/01/01,BUSINESS,Level 6,2022,07A345678,HU,Huye,huye_students.csv,2026-02-12 09:44:47.451886
2,HU003,alice,NaN,01-02-2001,Engineering,level 7,2021,0781234567,HU,Huye,huye_students.csv,2026-02-12 09:44:47.451886
3,HU004,charles#,NaN,2003-05-12,BUSINESS,Level 6,2021,07A345678,HU,Huye,huye_students.csv,2026-02-12 09:44:47.451886
4,HU005,charles#,Male,2000/01/01,BUSINESS,level 7,2020,0781234567,HU,Huye,huye_students.csv,2026-02-12 09:44:47.451886


In [50]:
results.head()

,Student_ID,Course_Code,Assessment_Type,Mark,Assessment_Date,Academic_Year,Semester,Attendance_Rate
0,MU011,CS102,Test,110.0,10/02/2023,2023/2024,Semester 1,105.0
1,HU008,CS101,Assignment,110.0,2023/03/15,2023/2024,Semester 1,40.0
2,MU013,CS-101,Test,-5.0,2023-01-10,2023/2024,Semester 1,40.0
3,KI033,CS101,Assignment,60.0,10/02/2023,2023/2024,Semester1,105.0
4,HU042,CS101,Assignment,75.0,2023/03/15,2023/2024,Semester 1,105.0


In [49]:
courses.head()

,Course_Code,Course_Title,Credits
0,CS101,Python,3
1,CS-101,Python,3
2,CS102,Databases,4
3,cs103,AI,-2


DATA PROFILING

In [29]:
print("Missing values:\n", students.isnull().sum())
print("\nDuplicates in students:", students.duplicated().sum())


Missing values:
 Student_ID             0
Full_Name              0
Gender                45
DOB                   36
Program                0
Level                  0
Intake_Year            0
Phone                 50
Campus_ID              0
Campus_Name            0
Source_Campus_File     0
Upload_Date            0
dtype: int64

Duplicates in students: 0


CLEANING: MISSING VALUES

In [55]:
students["Gender"] = students["Gender"].fillna("Unknown")
students["Phone"] = students["Phone"].fillna("Not Provided")

results["Mark"] = pd.to_numeric(results["Mark"], errors="coerce")
results["Mark"] = results["Mark"].fillna(results["Mark"].median())

results["Attendance_Rate"] = results["Attendance_Rate"].fillna(results["Attendance_Rate"].mean())

students["DOB"] = students["DOB"].fillna("Unknown")


In [56]:
print("Missing values:\n", students.isnull().sum())
print("\nDuplicates in students:", students.duplicated().sum())

Missing values:
 Student_ID            0
Full_Name             0
Gender                0
DOB                   0
Program               0
Level                 0
Intake_Year           0
Phone                 0
Campus_ID             0
Campus_Name           0
Source_Campus_File    0
Upload_Date           0
dtype: int64

Duplicates in students: 0


REMOVE DUPLICATES

In [31]:
students = students.drop_duplicates(subset=["Student_ID"])
results = results.drop_duplicates(subset=["Student_ID","Course_Code","Assessment_Type"])


In [52]:
#detecting outliers
def detect_outliers_iqr(df, column):
    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    # Calculate the Interquartile Range
    IQR = Q3 - Q1
    
    # Define bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify the outlier rows
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    print(f"--- Outlier Detection for: {column} ---")
    print(f"Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
    print(f"Lower Bound: {lower_bound:.2f}, Upper Bound: {upper_bound:.2f}")
    print(f"Number of outliers detected: {len(outliers)}")
    
    if not outliers.empty:
        print("\nIdentified Outlier Rows:")
        print(outliers[['Student_ID', 'Course_Code', column]])
    else:
        print("\nNo outliers found in this column.")
    
    return outliers

# Example usage on your gold dataframe:
outliers_mark = detect_outliers_iqr(gold, 'Mark')
print("\n" + "="*40 + "\n")
outliers_attendance = detect_outliers_iqr(gold, 'Attendance_Rate')

--- Outlier Detection for: Mark ---
Q1: 60.00, Q3: 60.00, IQR: 0.00
Lower Bound: 60.00, Upper Bound: 60.00
Number of outliers detected: 26

Identified Outlier Rows:
    Student_ID Course_Code  Mark
9        HU006      CS-101  75.0
53       HU026      CS102   45.0
62       MU013      CS-101  45.0
84       MU038       CS101  45.0
85       MU038       CS101  45.0
86       MU030       CS101  45.0
87       MU030       CS101  45.0
109      KI012       cs103  45.0
122      KI014       CS101  75.0
123      KI014       CS101  75.0
132      HU030      CS-101  75.0
150      MU025       CS101  45.0
151      MU025       CS101  45.0
152      KI047      CS102   75.0
157      KI003      CS-101  45.0
164      MU035      CS102   75.0
167      MU011       cs103  75.0
173      KI036       CS101  45.0
174      KI036       CS101  45.0
179      MU023       cs103  45.0
189      HU015      CS102   75.0
202      HU043       cs103  45.0
206      KI024      CS102   75.0
208      HU038       cs103  75.0
223      K

HANDLE OUTLIERS

In [32]:
#using median imputation to handle outliers and invalid data
results.loc[(results["Mark"] < 0) | (results["Mark"] > 100), "Mark"] = np.nan
results["Mark"] = results["Mark"].fillna(results["Mark"].median())

results.loc[results["Attendance_Rate"] > 100, "Attendance_Rate"] = 100

courses["Credits"] = courses["Credits"].apply(lambda x: x if x > 0 else 3)


STANDARDIZE FORMATS

In [33]:
students["Full_Name"] = students["Full_Name"].str.strip().str.title()
students["Full_Name"] = students["Full_Name"].str.replace(r'[^a-zA-Z ]','', regex=True)

students["Program"] = students["Program"].str.strip().str.upper()
students["Level"] = students["Level"].str.replace(" ","").str.title()

courses["Course_Code"] = courses["Course_Code"].str.replace("-","").str.strip().str.upper()

results["Assessment_Date"] = pd.to_datetime(results["Assessment_Date"], errors="coerce")
results["Semester"] = results["Semester"].str.replace(" ","")


TRANSFORMATION

In [64]:
# Scaling attendance
scaler = StandardScaler()
results["Attendance_Scaled"] = scaler.fit_transform(results[["Attendance_Rate"]])

results["Mark"] = results["Mark"].clip(lower=0.00, upper=100)

# Binning Marks
bins = [0,49,59,69,100]
labels = ["Fail","Pass","Credit","Distinction"]
results["Grade"] = pd.cut(results["Mark"], bins=bins, labels=labels)
results.head(2)

,Student_ID,Course_Code,Assessment_Type,Mark,Assessment_Date,Academic_Year,Semester,Attendance_Rate,Attendance_Scaled,Grade
0,MU011,CS102,Test,100.0,10/02/2023,2023/2024,Semester 1,105.0,1.560991,Distinction
1,HU008,CS101,Assignment,100.0,2023/03/15,2023/2024,Semester 1,40.0,-1.414787,Distinction


Save silver

In [35]:
results.to_csv("silver_transformed.csv", index=False)

INTEGRATION (CREATE GOLD)

In [36]:
gold = results.merge(students, on="Student_ID", how="left")
gold = gold.merge(courses, on="Course_Code", how="left")

gold.to_csv("gold_integrated.csv", index=False)
gold.head()


,Student_ID,Course_Code,Assessment_Type,Mark,Assessment_Date,Academic_Year,Semester,Attendance_Rate,Attendance_Scaled,Grade,...,Program,Level,Intake_Year,Phone,Campus_ID,Campus_Name,Source_Campus_File,Upload_Date,Course_Title,Credits
0,MU011,CS102,Test,60.0,2023-10-02,2023/2024,Semester1,100.0,1.499738,Credit,...,ICT,Level6,2022,Not Provided,MU,Musanze,musanze_students.csv,2026-02-12 09:44:47.474890,NaN,NaN
1,HU008,CS101,Assignment,60.0,NaT,2023/2024,Semester1,40.0,-1.469301,Credit,...,ICT,Level6,2022,Not Provided,HU,Huye,huye_students.csv,2026-02-12 09:44:47.451886,Python,3.0
2,HU008,CS101,Assignment,60.0,NaT,2023/2024,Semester1,40.0,-1.469301,Credit,...,ICT,Level6,2022,Not Provided,HU,Huye,huye_students.csv,2026-02-12 09:44:47.451886,Python,3.0
3,MU013,CS-101,Test,60.0,NaT,2023/2024,Semester1,40.0,-1.469301,Credit,...,ENGINEERING,Level6,2020,Not Provided,MU,Musanze,musanze_students.csv,2026-02-12 09:44:47.474890,NaN,NaN
4,KI033,CS101,Assignment,60.0,2023-10-02,2023/2024,Semester1,100.0,1.499738,Credit,...,BUSINESS,Level6,2022,07A345678,KI,Kigali,kigali_students.csv,2026-02-12 09:44:47.466967,Python,3.0


FEATURE ENGINEERING

In [37]:
# Date features
gold["assessment_month"] = gold["Assessment_Date"].dt.month
gold["assessment_weekday"] = gold["Assessment_Date"].dt.day_name()
gold["is_weekend_assessment"] = gold["Assessment_Date"].dt.weekday >= 5

# Aggregations
agg = gold.groupby("Student_ID").agg(
    student_course_count=("Course_Code","nunique"),
    student_avg_mark=("Mark","mean"),
    student_max_mark=("Mark","max"),
    student_fail_count=("Grade",lambda x:(x=="Fail").sum()),
    student_total_credits_earned=("Credits","sum")
).reset_index()

gold = gold.merge(agg, on="Student_ID")

# Risk flags
gold["is_at_risk"] = np.where(
    (gold["student_avg_mark"] < 50) | (gold["student_fail_count"] >= 2),
    1,0
)

gold["low_attendance_flag"] = np.where(gold["Attendance_Rate"] < 60,1,0)

gold.to_csv("gold_features.csv", index=False)


REQUIRED CHARTS

In [47]:
gold_integrated = pd.read_csv("gold_integrated.csv")

# Quick check
gold_integrated.head()
# Check missing values
gold_integrated.isnull().sum()


Student_ID            0
Course_Code           0
Assessment_Type       0
Mark                  0
Assessment_Date       0
Academic_Year         0
Semester              0
Attendance_Rate       0
Attendance_Scaled     0
Grade                 0
Full_Name             0
Gender                0
DOB                   0
Program               0
Level                 0
Intake_Year           0
Phone                 0
Campus_ID             0
Campus_Name           0
Source_Campus_File    0
Upload_Date           0
Course_Title          0
Credits               0
dtype: int64

HANDLING MISSING VALUES FOUND

In [43]:
# 1. Load the integrated gold data
gold = pd.read_csv("gold_integrated.csv")

# 2. Handle missing values
# Drop rows where Assessment_Date is missing (critical for the next cell's logic)
gold = gold.dropna(subset=['Assessment_Date'])

# Fill DOB, Course_Title, and Credits with logical defaults
gold["DOB"] = gold["DOB"].fillna("Unknown")
gold["Course_Title"] = gold["Course_Title"].fillna("Unknown Course")
gold["Credits"] = gold["Credits"].fillna(0)

# 3. Ensure Assessment_Date is converted back to datetime objects 
# (Pandas reads them as strings from CSV by default)
gold["Assessment_Date"] = pd.to_datetime(gold["Assessment_Date"])

# Verify the fix
# 4. PRINT FINAL VERIFICATION (As requested)
print("--- FINAL MISSING VALUES CHECK ---")
print(gold[['Assessment_Date', 'DOB', 'Course_Title', 'Credits']].isnull().sum())
print(f"\nTotal clean rows remaining: {len(gold)}")

# 5. SAVE the final version (overwriting the old one with the clean data)
gold.to_csv("gold_integrated.csv", index=False)
print("\nSuccess: 'gold_integrated.csv' is now clean and saved.")

--- FINAL MISSING VALUES CHECK ---
Assessment_Date    0
DOB                0
Course_Title       0
Credits            0
dtype: int64

Total clean rows remaining: 93

Success: 'gold_integrated.csv' is now clean and saved.


In [53]:
gold.head()

,Student_ID,Course_Code,Assessment_Type,Mark,Assessment_Date,Academic_Year,Semester,Attendance_Rate,Attendance_Scaled,Grade,...,Program,Level,Intake_Year,Phone,Campus_ID,Campus_Name,Source_Campus_File,Upload_Date,Course_Title,Credits
0,MU011,CS102,Test,60.0,2023-10-02,2023/2024,Semester1,100.000000,1.499738,Credit,...,ICT,Level6,2022,Not Provided,MU,Musanze,musanze_students.csv,2026-02-12 09:44:47.474890,Unknown Course,0.0
4,KI033,CS101,Assignment,60.0,2023-10-02,2023/2024,Semester1,100.000000,1.499738,Credit,...,BUSINESS,Level6,2022,07A345678,KI,Kigali,kigali_students.csv,2026-02-12 09:44:47.466967,Python,3.0
5,KI033,CS101,Assignment,60.0,2023-10-02,2023/2024,Semester1,100.000000,1.499738,Credit,...,BUSINESS,Level6,2022,07A345678,KI,Kigali,kigali_students.csv,2026-02-12 09:44:47.466967,Python,3.0
9,HU006,CS-101,Exam,75.0,2023-10-02,2023/2024,Semester1,80.000000,0.510058,Distinction,...,ICT,Level7,2021,0781234567,HU,Huye,huye_students.csv,2026-02-12 09:44:47.451886,Unknown Course,0.0
10,HU026,cs103,Exam,60.0,2023-10-02,2023/2024,Semester1,70.903226,0.059913,Credit,...,BUSINESS,Level6,2022,07A345678,HU,Huye,huye_students.csv,2026-02-12 09:44:47.451886,Unknown Course,0.0
